Mount Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Read Data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame
import seaborn as sns
import sklearn
import sklearn.metrics as metrics
from keras.backend import clear_session
from keras.layers import Dense
from keras.models import Sequential
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
# from sklearn.svm import SVR
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold 
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import joblib

def scale_datasets(X_train, y_train):
  """
  Standard Scale test and train data
  Z - Score normalization
  """
  std_pred, std_resp = StandardScaler(), StandardScaler()
  X_train, y_train = std_pred.fit_transform(X_train), std_resp.fit_transform(y_train.to_numpy().reshape(-1, 1))
  return X_train, y_train

  
df = pd.read_csv('./drive//MyDrive//Data//collected_us.csv')
df = df[df['kickoff_time'] <= '2020-07-26'] # Note: Look at me
df.drop(['player_name', 'GW', 'kickoff_time'], axis=1, inplace=True)
std_pred, std_resp = StandardScaler(), StandardScaler()
X_train, y_train = df.select_dtypes(include = 'number').drop('total_points', axis=1), df['total_points'] # NOTE: look at me
X_train, y_train = X_train.iloc[0:200, 0:10], y_train.iloc[0:200]

Random Forest Parameter Tuning

In [ ]:
rf_n_estimators = [int(x) for x in np.linspace(200, 1000, 5)]
rf_n_estimators.append(1500)
rf_n_estimators.append(2000)
rf_max_depth = [int(x) for x in np.linspace(5, 55, 11)]
rf_max_depth.append(None)
rf_max_features = ['auto', 'sqrt', 'log2']
rf_criterion = ['mse', 'mae']
rf_min_samples_split = [int(x) for x in np.linspace(2, 10, 9)]
rf_min_impurity_decrease = [0.0, 0.05, 0.1]
rf_bootstrap = [True, False]
rf_grid = {'n_estimators': rf_n_estimators,
               'max_depth': rf_max_depth,
               'max_features': rf_max_features,
               'criterion': rf_criterion,
               'min_samples_split': rf_min_samples_split,
               'min_impurity_decrease': rf_min_impurity_decrease,
               'bootstrap': rf_bootstrap}
rf_base = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf_base, param_distributions = rf_grid, 
                               n_iter = 10, cv = 3, verbose = 2, random_state = 42, 
                               n_jobs = -1)
rf_random.fit(X_train, y_train)
joblib.dump(rf_random, 'random_forst_search.pkl')

In [ ]:
rf_results = joblib.load('./random_forst_search.pkl')
rf_results.best_estimator_

Support Vector Regression Parameter Tuning

In [ ]:
! pip install thundersvm
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls  # Check if required cuda 9.0 amd64-deb file is downloaded
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls /var/cuda-repo-9-0-local | grep .pub
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!sudo apt-get install cuda-9.0
from thundersvm import SVR

In [ ]:
from thundersvm import SVR
import multiprocessing
if __name__ == '__main__':
  df = pd.read_csv('./drive//MyDrive//Data//collected_us.csv')
  df = df[df['kickoff_time'] <= '2020-07-26'] # Note: Look at me
  df.drop(['player_name', 'GW', 'kickoff_time'], axis=1, inplace=True)
  std_pred, std_resp = StandardScaler(), StandardScaler()
  X_train, y_train = df.select_dtypes(include = 'number').drop('total_points', axis=1), df['total_points'] # NOTE: look at me
  X_train, y_train = std_pred.fit_transform(X_train.iloc[0:200, 0:10]), std_resp.fit_transform(y_train.iloc[0:200].to_numpy().reshape(-1, 1))


  hyperparameters = {'gamma': [float(x) for x in np.linspace(0.0, 0.3, 5)], 
                      'C': [float(x) for x in np.linspace(0.0, 1000, 5)],
                    'epsilon': [float(x) for x in np.linspace(0.0, 1, 5)],
                    'kernel': ['rbf', 'poly', 'linear']}
  grid = RandomizedSearchCV(estimator = SVR(verbose=2),param_distributions = hyperparameters,
                            n_iter = 10, cv = 3,  verbose = 2, random_state = 42, n_jobs=8)
  grid.fit(X_train,y_train)
  print(grid.best_estimator_)
  print(grid.best_score_)
  joblib.dump(grid, 'svr.pkl')


Keras Regressor

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

def tune_nn_model(hp, MAX_LAYERS = 2):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 1, MAX_LAYERS)):

      units = hp.Int('units_' + str(i),
                     min_value=32,
                     max_value=512,
                     step=120,
                     default = 512)
      
      model.add(keras.layers.Dense(units=units, activation='relu',
                                   kernel_initializer=tf.keras.initializers.HeUniform()))

      drop_rate = hp.Choice('drop_rate_' + str(i), [0.0, 0.1, 0.3],default = 0)
      model.add(keras.layers.Dropout(rate=drop_rate))

      l1_reg = hp.Choice('l1_reg_' + str(i), [0.0, 0.01, 0.001],default = 0)
      model.add(keras.layers.ActivityRegularization(l1 = l1_reg))
      
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'SGD', 'rmsprop'])
    optimizer = tf.keras.optimizers.get(hp_optimizer)
    optimizer.learning_rate = hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4], default=0.01)

    model.add(keras.layers.Dense(1, activation='linear',kernel_initializer=tf.keras.initializers.HeUniform()))
    model.compile(optimizer=optimizer, loss = 'mse', metrics = ['mse'])

    return model

MAX_TRIALS = 1
MAX_EXECUTIONS = 1
EPOCHS = 1
MAX_LAYERS = 2

b_tuner = kt.BayesianOptimization(
    tune_nn_model,
    objective='mse',
    max_trials=MAX_TRIALS,
    executions_per_trial=MAX_EXECUTIONS,
    directory='test_dir',
    project_name='b_tune_nn',
    seed=0,
    overwrite = True,
    num_initial_points=2, alpha=0.0001, beta=2.6
)

X_train, y_train = df.select_dtypes(include = 'number').drop('total_points', axis=1), df['total_points'] # NOTE: look at me
X_train, y_train = scale_datasets(X_train, y_train)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
b_tuner.search(X_train[0:100], y_train[0:100], epochs=EPOCHS, validation_split=0.2, callbacks=[stop_early])
joblib.dump(b_tuner, 'neural_net_bayes.pkl')

In [ ]:
tuner = joblib.load('./neural_net_bayes.pkl')
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps) 
print(model)
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2) # Note this should be long enough
val_acc_per_epoch = history.history['val_loss']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))


def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [Total points]')
  plt.legend()
  plt.grid(True)
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()
plot_loss(history)


In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR

X_train, y_train = df.select_dtypes(include = 'number').drop('total_points', axis=1), df['total_points'] 
X_train, y_train = X_train.iloc[0:100,0:5], y_train.iloc[0:100]
X_train, y_train = scale_datasets(X_train, y_train)


param_grid = {
    'base_estimator__C' : [0.1, 1, 10, 100],
    'base_estimator__gamma' : [0.1, 1, 10, 100],
    'base_estimator__epsilon' : [0.1, 1, 10, 100],
    'n_estimators' : [10, 50, 100],
    'bootstrap' : [True, False]
    }



bagged_svr = GridSearchCV(BaggingRegressor(SVR()), param_grid, n_jobs= 4, cv = 3,verbose = 3).fit(X_train, y_train)
joblib.dump(bagged_svr, 'bagged_svr.pkl')


In [ ]:
bagged_svr.best_estimator_

kNN Regression


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
X_train, y_train = df.select_dtypes(include = 'number').drop('total_points', axis=1), df['total_points'] 
# X_train, y_train = X_train.iloc[0:100,0:5], y_train.iloc[0:100]
X_train, y_train = scale_datasets(X_train, y_train)

estimator_KNN = KNeighborsRegressor(algorithm='kd_tree')

parameters_KNN = {
    'n_neighbors': (1,12, 1),
    'leaf_size': (20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev')}
                   
# with GridSearch
grid_search_KNN = GridSearchCV(estimator=estimator_KNN,
    param_grid=parameters_KNN, n_jobs= 4, cv = 5, verbose = 3).fit(X_train, y_train)
grid_search_KNN.best_estimator_

In [ ]:
grid_search_KNN.cv_results_

Linear Regression

In [ ]:
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid_lr = GridSearchCV(LinearRegression(), parameters, cv=5).fit(X_train, y_train)
grid_lr.best_estimator_